In [4]:
# !pip install mlxtend

In [34]:
!pip install xgboost

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import SVR
# from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
# from mlxtend.Regressor import StackingCVRegressor #
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('datasets_52721_99691_student-mat.csv')


In [6]:
X = df.drop('G3', axis=1)
y = df[['G3']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
172,GP,M,17,U,LE3,T,4,4,teacher,other,reputation,mother,1,2,0,no,yes,yes,yes,yes,yes,yes,no,4,4,4,1,3,5,0,13,11
119,GP,M,15,U,GT3,T,3,4,other,other,reputation,father,1,1,0,no,no,no,no,yes,yes,yes,no,3,4,3,1,2,4,6,14,13
389,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,1,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,0,6,5
198,GP,F,17,U,GT3,T,4,4,services,teacher,home,mother,2,1,1,no,yes,no,no,yes,yes,yes,no,4,2,4,2,3,2,24,18,18
139,GP,F,15,U,GT3,T,4,4,teacher,teacher,course,mother,2,1,0,no,no,no,yes,yes,yes,yes,no,4,3,2,1,1,5,0,16,16


In [49]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

num_transform = Pipeline(steps=[('scaler', StandardScaler())])

cat_transform = Pipeline(steps=[('oneHot', OneHotEncoder(handle_unknown='ignore'))])

In [53]:
cat_features

Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic'],
      dtype='object')

In [111]:
num_features = df.select_dtypes(include=['int64', 'float64']).drop(['G3'], axis=1).columns
cat_features = df.select_dtypes(include=['object']).columns
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat',  OneHotEncoder(handle_unknown='ignore'), cat_features)])

# preprocessor = make_column_transformer(
#         (StandardScaler(), num_features),
#         (OneHotEncoder(handle_unknown='ignore'), cat_features))

# imputer_cat_pipeline = make_column_transformer(
#     (make_pipeline(SimpleImputer(strategy='constant'), cat_columns_fill_miss),
#     (make_pipeline(SimpleImputer(strategy='most_frequent'), cat_columns_fill_freq),
# )

# encoder_cat_pipeline = make_column_transformer(
#     (OrdinalEncoder(categories=ord_mapping), cat_columns_ord),
#     (OneHotEncoder(), cat_columns_onehot),
# )


In [113]:

from sklearn.linear_model import LinearRegression

rf = RandomForestRegressor(random_state = 1000, n_jobs = -1)
lr = LinearRegression(n_jobs = -1)




In [157]:
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt',0.5]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3, 5,7]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
params = {
               'stackingcvregressor__randomforestregressor__n_estimators': n_estimators,  #randomforestregressor__bootstrap
               'stackingcvregressor__randomforestregressor__max_features': max_features,
               'stackingcvregressor__randomforestregressor__max_depth': max_depth,
               'stackingcvregressor__randomforestregressor__min_samples_split': min_samples_split,
               'stackingcvregressor__randomforestregressor__min_samples_leaf': min_samples_leaf,
               'stackingcvregressor__randomforestregressor__bootstrap': bootstrap,
#               "xgbclassifier__n_estimators": [10, 50, 100, 500],
#                 "xgbclassifier__learning_rate": [0.1, 0.5, 1],
         }
print(params)

{'stackingcvregressor__randomforestregressor__n_estimators': [20, 73, 126, 180, 233, 286, 340, 393, 446, 500], 'stackingcvregressor__randomforestregressor__max_features': ['auto', 'sqrt', 0.5], 'stackingcvregressor__randomforestregressor__max_depth': [5, 15, 26, 36, 47, 57, 68, 78, 89, 99, 110, None], 'stackingcvregressor__randomforestregressor__min_samples_split': [2, 5, 10], 'stackingcvregressor__randomforestregressor__min_samples_leaf': [1, 3, 5, 7], 'stackingcvregressor__randomforestregressor__bootstrap': [True, False]}


In [156]:
# pipe_stack.get_params()

In [ ]:
# rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', rf)])

# rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', c3)])

stacking_model = StackingCVRegressor(regressors=(rf,lr), 
                            meta_regressor=lr,
                            random_state=42,use_features_in_secondary=True)

pipe_stack = make_pipeline(preprocessor, stacking_model)
# pipe_stack = Pipeline([('preprocessor', preprocessor), ('model', stacking_model)])

grid = GridSearchCV(estimator=pipe_stack, 
                    param_grid=params, 
                    cv=3,
                    refit=True)
grid.fit(X_train, y_train)



In [142]:
rf_pipe = Pipeline(steps=[('classifier', rf)])

# rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', c3)])

stacking_model = StackingCVRegressor(regressors=(rf,xgb()), 
                            meta_regressor=lr,
                            random_state=42,use_features_in_secondary=True)

# pipe_stack = make_pipeline(preprocessor, stacking_model)
pipe_stack = Pipeline([('preprocessor', preprocessor), ('model', stacking_model)])

# pipe = Pipeline([
#     ('preprocessor', preprocessor),('stack',stacking_model)
# ])
# params = {'kneighborsclassifier__n_neighbors': [1, 5],
#           'randomforestclassifier__n_estimators': [10, 50],
#           'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=pipe_stack, 
                    param_grid=params, 
                    cv=3,
                    refit=True)
grid.fit(X_train, y_train)



TypeError: 'module' object is not callable

In [ ]:
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
print('Accuracy: %.2f' % grid.best_score_)